In [29]:
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

## Load test dataset

In [30]:
test_df = pd.read_csv('../Splits/combined/sub_test.csv')
# test_df = pd.read_csv('../Splits/harder_test/sub_test.csv')
test_df

,Unnamed: 0,writer,problem_id,submission_id,website,source,dataset_type
0,0,Human,NaN,13688015.0,https://codeforces.com,#include<iostream>\n#include<cstdio>\n#include...,test
1,1,Human,NaN,36590979.0,https://codeforces.com,#include <bits/stdc++.h>\n\nusing namespace st...,test
2,2,AI,1909G,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,test
3,3,Human,toi7_sewer,305644.0,https://programming.in.th,// YoruoniVamp - VTUBE\n// Pragma Credit to Di...,test
4,4,Human,NaN,36539175.0,https://codeforces.com,#include <bits/stdc++.h>\n// #include <ext/rop...,test
...,...,...,...,...,...,...,...
2395,2395,Human,NaN,35426753.0,https://codeforces.com,/**\n * author: skmonir\n * created: 17...,test
2396,2396,AI,1976F,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,test
2397,2397,AI,272D,NaN,https://codeforces.com,#include <bits/stdc++.h>\nusing namespace std;...,test
2398,2398,Human,1112,103903.0,https://programming.in.th,#include<bits/stdc++.h>\n\n\nusing namespace s...,test


In [31]:
test_df['source'][0]

'#include<iostream>\n#include<cstdio>\n#include<cstring>\n#include<algorithm>\n#include<cmath>\nusing namespace std;\nconst int N=1111;\nstruct node\n{\n int t,s,f,flag;\n}a[N];\nint main()\n{\n int n;\n cin>>n;\n for(int i=1;i<=n;++i)\n {\n  cin>>a[i].t>>a[i].s>>a[i].f;\n  a[i].flag=0;\n  if(a[i].s>a[i].f)a[i].flag=1;\n }\n for(int i=1;i<=n;++i)\n {\n  int ans=0;\n  for(int j=1;j<=n;++j)\n  {\n   if(i==j)continue;\n   if(a[i].flag==a[j].flag)\n   {\n    if(a[i].t==a[j].t)\n    {\n     if(a[i].s==a[j].s)ans++;\n     continue;\n    }\n    int tmp=0;\n    if(a[i].t>a[j].t)tmp=1;\n    if(tmp)swap(a[i],a[j]);\n    if(a[i].flag==0)\n    {\n     int p=a[i].s+a[j].t-a[i].t;\n     if(p<=a[i].f&&p==a[j].s)ans++;\n    }\n    else\n    {\n     int p=a[i].s-(a[j].t-a[i].t);\n     if(p>=a[i].f&&p==a[j].s)ans++;\n    }\n    if(tmp)swap(a[i],a[j]);\n   }\n   else \n   {\n    int tmp=0;\n    if(a[i].t>a[j].t)tmp=1;\n    if(tmp)swap(a[i],a[j]);\n    int ok=0;\n    if(a[i].flag==0)\n    {\n     int p=a[

In [32]:
url = "http://localhost:59875/analyze"
response = requests.post(url, json={"code":test_df['source'][0]})
" ".join(json.loads(response.text)['details'])

'Consistent formatting detected (uniform indentation), which is often a trait of AI-generated code. Specific, context-aware variable names detected, which is more typical of human-written code. No comments found in the code. AI-generated code often lacks comments. Repetitive patterns found in the code, which can be a sign of AI generation. Overly structured code detected (e.g., repeated if-else or loops), which might indicate AI involvement. Simpler code detected, which could indicate AI generation.'

In [33]:
responses = []
err_logs = []
url = "http://localhost:59875/analyze"
from rich import print

from IPython.display import clear_output
for index, row in test_df.iterrows():
  source = row['source']
  print(f"ASKED : {index}")
  try:
    response = requests.post(url, json={"code":source})
    clear_output()
    data = json.loads(response.text)
    responses.append(data)
  except Exception as e:
    clear_output()
    err_logs.append((index,e))
    print(f"ERROR at {index} : {e}")

  

In [34]:
err_logs

[]

In [35]:
import re
import json

result_n = len(responses)
answer_df = test_df.iloc[:result_n]
answer_df['answer_text'] = responses

C:\Users\Pakin\AppData\Local\Temp\ipykernel_31520\2693926148.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['answer_text'] = responses


In [36]:
answer_df['model_answer'] = answer_df['answer_text'].apply(lambda x: x['evaluate'])
answer_df['model_reason'] = answer_df['answer_text'].apply(lambda x: " ".join(x['details']))
answer_df['verdict'] = answer_df['writer'] == answer_df['model_answer']
answer_df = answer_df[['writer','model_answer','verdict','model_reason','website','source']]

C:\Users\Pakin\AppData\Local\Temp\ipykernel_31520\319285290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['model_answer'] = answer_df['answer_text'].apply(lambda x: x['evaluate'])
C:\Users\Pakin\AppData\Local\Temp\ipykernel_31520\319285290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer_df['model_reason'] = answer_df['answer_text'].apply(lambda x: " ".join(x['details']))
C:\Users\Pakin\AppData\Local\Temp\ipykernel_31520\319285290.py:3: SettingWithCopyWarning: 
A value is trying to

In [37]:
filtered_df = pd.DataFrame()
filtered_df[['Label','Model_Answer','Verdict','Reason','Website']] = answer_df[['writer','model_answer','verdict','model_reason','website']]

In [38]:
filtered_df['Label'] = filtered_df['Label'].replace('Human', 'human')
filtered_df['Model_Answer'] = filtered_df['Model_Answer'].replace('Human', 'human')
filtered_df['Verdict'] = filtered_df['Label']==filtered_df['Model_Answer']

In [39]:
filtered_df['Label'].value_counts(), filtered_df['Model_Answer'].value_counts()

(Label
 human    1200
 AI       1200
 Name: count, dtype: int64,
 Model_Answer
 AI       1663
 human     737
 Name: count, dtype: int64)

In [40]:
filtered_df.to_csv("./Results/Normal/Result-RuleBased.csv",index=False)

In [41]:
accuracy = filtered_df['Verdict'].value_counts(normalize=True).get(True, 0)
print(f"AUC : {accuracy}")

AUC : 0.56375